# Example usage notebook
Objects and functions in this notebook are listed with all paramaters to ilustrate their capabillities. Most of the paramaters have default values in the implementation

## Imports

In [1]:
import os
import sys

sys.path.append(os.getcwd())
os.chdir("..")

import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 300)

In [2]:
import os
import json
from typing import Literal
import pandas as pd

from churn_pred.preprocessing.preprocess import PreprocessData
from churn_pred.training.trainer import Trainer
from churn_pred.training.optuna_optimizer import LGBOptunaOptimizer

import dill
import numpy as np
from churn_pred.utils import dill_dump
from churn_pred.utils import dill_load
from sklearn.model_selection import train_test_split

from pprint import pprint

## Train a testing model 

In [3]:
# 1. get the data
df_pd = pd.read_parquet("data/dataset_auxiliary_features_cleaned.parquet")
df_pd.head()

,CustomerId,CreditScore,Country,Gender,Age,Tenure,Balance (EUR),NumberOfProducts,HasCreditCard,IsActiveMember,EstimatedSalary,Exited,CustomerFeedback_sentiment3,CustomerFeedback_sentiment5,Surname_Country,Surname_Country_region,Surname_Country_subregion,Country_region,Country_subregion,is_native,Country_hemisphere,Country_gdp_per_capita,Country_IncomeGroup,Surname_Country_gdp_per_capita,Surname_Country_IncomeGroup,working_class,stage_of_life,generation
0,15787619,844,France,Male,18,2,160980.03,1,0,0,145936.28,0,neutral,4 stars,Taiwan,Asia,Eastern Asia,Europe,Western Europe,0,northern,57594.03402,High income,32756.00000,None,working_age,teen,gen_z
1,15770309,656,France,Male,18,10,151762.74,1,0,1,127014.32,0,neutral,1 star,United States,Americas,Northern America,Europe,Western Europe,0,northern,57594.03402,High income,76329.58227,High income,working_age,teen,gen_z
2,15569178,570,France,Female,18,4,82767.42,1,1,0,71811.90,0,neutral,2 stars,Russian Federation,Europe,Eastern Europe,Europe,Western Europe,0,northern,57594.03402,High income,34637.76172,Upper middle income,working_age,teen,gen_z
3,15795519,716,Germany,Female,18,3,128743.80,1,0,0,197322.13,0,neutral,2 stars,Russian Federation,Europe,Eastern Europe,Europe,Western Europe,0,northern,66616.02225,High income,34637.76172,Upper middle income,working_age,teen,gen_z
4,15621893,727,France,Male,18,4,133550.67,1,1,1,46941.41,0,positive,1 star,Italy,Europe,Southern Europe,Europe,Western Europe,0,northern,57594.03402,High income,55442.07843,High income,working_age,teen,gen_z


In [4]:
target_col = "Exited"
id_cols = ["CustomerId"]
cat_cols = [
    "Country",
    "Gender",
    "HasCreditCard",
    "IsActiveMember",
    "CustomerFeedback_sentiment3",
    "CustomerFeedback_sentiment5",
    "Surname_Country",
    "Surname_Country_region",
    "Surname_Country_subregion",
    "Country_region",
    "Country_subregion",
    "is_native",
    "Country_hemisphere",
    "Country_IncomeGroup",
    "Surname_Country_IncomeGroup",
    "working_class",
    "stage_of_life",
    "generation",
]
cont_cols = df_pd.drop(
    columns=id_cols + cat_cols + [target_col]
).columns.values.tolist()

In [5]:
df_pd[cat_cols] = df_pd[cat_cols].astype(str)

In [6]:
valid_size = 0.2
test_size = 0.5
random_state = 1
df_train, df_valid = train_test_split(
    df_pd, test_size=valid_size, stratify=df_pd[target_col], random_state=random_state
)
df_valid, df_test = train_test_split(
    df_valid,
    test_size=test_size,
    stratify=df_valid[target_col],
    random_state=random_state,
)

In [7]:
prepare_data = PreprocessData(
    id_cols=id_cols,
    target_col=target_col,
    cat_cols=cat_cols,
    cont_cols=cont_cols,
)
# this should be fitted only on training data
_ = prepare_data.fit(df=df_pd)

In [8]:
optimizer = LGBOptunaOptimizer(
    objective="binary",
    n_class=2,
)

trainer = Trainer(
    cat_cols=prepare_data.cat_cols,
    target_col=prepare_data.target_col,
    id_cols=id_cols,
    objective="binary",
    n_class=2,
    optimizer=optimizer,
    preprocessors=[prepare_data],
)

metrics_dict = trainer.fit(
    df_train=df_train,
    df_valid=df_valid,
    df_test=df_test,
)

[I 2024-05-01 21:23:45,400] A new study created in memory with name: no-name-d630027d-067f-416b-b1eb-7c87a894bc0a
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.379844:  14%|#4        | 1/7 [00:07<00:43,  7.21s/it]

Early stopping, best iteration is:
[163]	valid_0's binary_logloss: 0.379844
Training until validation scores don't improve for 50 rounds


feature_fraction, val_score: 0.379844:  29%|##8       | 2/7 [00:09<00:21,  4.27s/it]

...

Early stopping, best iteration is:
[109]	valid_0's binary_logloss: 0.392082
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.363165:  80%|########  | 4/5 [00:11<00:02,  2.89s/it]

Early stopping, best iteration is:
[41]	valid_0's binary_logloss: 0.37577
Training until validation scores don't improve for 50 rounds


min_child_samples, val_score: 0.363165: 100%|##########| 5/5 [00:14<00:00,  2.86s/it]

Early stopping, best iteration is:
[43]	valid_0's binary_logloss: 0.374331


In [9]:
# save/load trainer
dill_dump("data/model/lgbm_trainer.dill", trainer)
trainer = dill_load("data/model/lgbm_trainer.dill")

In [10]:
pprint(metrics_dict)

{'cls_report': {'0': {'f1-score': 0.9143206854345166,
                      'precision': 0.8914081145584726,
                      'recall': 0.9384422110552764,
                      'support': 796},
                '1': {'f1-score': 0.6174863387978142,
                      'precision': 0.6975308641975309,
                      'recall': 0.553921568627451,
                      'support': 204},
                'accuracy': 0.86,
                'macro avg': {'f1-score': 0.7659035121161655,
                              'precision': 0.7944694893780018,
                              'recall': 0.7461818898413637,
                              'support': 1000},
                'weighted avg': {'f1-score': 0.8537664787206294,
                                 'precision': 0.8518571554848405,
                                 'recall': 0.86,
                                 'support': 1000}},
 'cm': [[747, 49], [91, 113]],
 'prec_rec_curve': [[0.204, 0.6975308641975309, 1.0],
                 

In [11]:
pprint(trainer.compute_metrics(df_test, with_dynamic_binary_threshold=True))

{'cls_report': {'0': {'f1-score': 0.9143920595533498,
                      'precision': 0.9031862745098039,
                      'recall': 0.9258793969849246,
                      'support': 796},
                '1': {'f1-score': 0.6443298969072165,
                      'precision': 0.6793478260869565,
                      'recall': 0.6127450980392157,
                      'support': 204},
                'accuracy': 0.862,
                'macro avg': {'f1-score': 0.7793609782302832,
                              'precision': 0.7912670502983803,
                              'recall': 0.7693122475120702,
                              'support': 1000},
                'weighted avg': {'f1-score': 0.8592993783735386,
                                 'precision': 0.8575232310315432,
                                 'recall': 0.862,
                                 'support': 1000}},
 'cm': [[737, 59], [79, 125]],
 'prec_rec_curve': [[0.204, 0.6793478260869565, 1.0],
              